In [229]:
import glob

In [233]:

output

['.\\1614221312_admin_my.txt', '.\\1614221328_admin_my.txt']

In [237]:
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Google Cloud Speech API samplbbbe application using the streaming API.

NOTE: This module requires the additional dependency `pyaudio`. To install
using pip:

    pip install pyaudio

Example usage:
    python transcribe_streaming_mic.py
"""

# [START speech_transcribe_streaming_mic]
from __future__ import division

import re
import sys

from google.cloud import speech

import pyaudio
from six.moves import queue
import time
import glob

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms


class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""

    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b"".join(data)


def listen_print_loop(responses,user_id):
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    """
    t = str(int(time.time()))
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        # The `results` list is consecutive. For streaming, we only care about
        # the first result being considered, since once it's `is_final`, it
        # moves on to considering the next utterance.
        result = response.results[0]
        if not result.alternatives:
            continue

        # Display the transcription of the top alternative.
        transcript = result.alternatives[0].transcript

        # Display interim results, but with a carriage return at the end of the
        # line, so subsequent lines will overwrite them.
        #
        # If the previous result was longer than this one, we need to print
        # some extra spaces to overwrite the previous result
        overwrite_chars = " " * (num_chars_printed - len(transcript))

        if not result.is_final:
                
            sys.stdout.write(transcript + overwrite_chars + "\r")
            sys.stdout.flush()

            num_chars_printed = len(transcript)

        else:
            print(transcript + overwrite_chars)
            
            with open("./"+t+"_"+user_id+"_my.txt","a") as f :
                f.writelines(transcript + overwrite_chars+"\n")
                
            # Exit recognition if any of the transcribed phrases could be
            # one of our keywords.
            if re.search(r"\b(exit|quit)\b", transcript, re.I):
                print("Exiting..")
                break

            num_chars_printed = 0


def main(user_id):
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = "ko-KR"  # a BCP-47 language tag

    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code
    )

    streaming_config = speech.StreamingRecognitionConfig(
        config=config, interim_results=True
    )

    global stream
    with MicrophoneStream(RATE, CHUNK) as stream:
        #print("step1")
        audio_generator = stream.generator()
        
        requests = (
            speech.StreamingRecognizeRequest(audio_content=content)
            for content in audio_generator
        )
        #print("step2")
        responses = client.streaming_recognize(streaming_config, requests)
        #print("step3")
        print(stream.closed)
        # stream.closed = True
        print(stream.closed)
        # Now, put the transcription responses to use.
        listen_print_loop(responses,user_id)
        #print("step4")
        # stream.closed = True

# if __name__ == "__main__":
#     main()
# [END speech_transcribe_streaming_mic]


In [28]:
!pip install konlpy

In [29]:
!pip install -U flask-cors

Requirement already up-to-date: flask-cors in c:\programdata\anaconda3\lib\site-packages (3.0.10)


In [ ]:
from flask import Flask,redirect,request,Response,send_file
from flask_cors import CORS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
import csv

okt = Okt()

app = Flask(__name__)
CORS(app)

@app.route('/user')
def user():
    user_id = request.args['id']
    main(user_id)
    # 이부분이 중단되었을때 코드를 작성하면 되겠죠
    
    return 'Hello, User!'

@app.route('/end')
def end():
    stream.closed = True
    return 'end'

@app.route('/txt')
def txt():
    print('step1')
    
    user_id = request.args['id']
    
    output = glob.glob('./*'+user_id+'_my.txt')
    output = sorted(output)
    
    f1 = open(output[-1],'r',encoding='euc-kr')
    txt = f1.readlines()
    f1.close()
    print(txt)
    morph_list = []
    for i in txt :
        morphs = okt.morphs(i)
        for morph in morphs:
            morph_list.append(morph)
        
    def counter(morph_list):
        word_count = {}
        for word in morph_list:
            if word in word_count :
                word_count[word] += 1
            else :
                word_count[word] = 1
        return word_count

    word_count = counter(morph_list)
    
    # 정렬
    word_count = sorted(word_count.items(), key = lambda x:x[1], reverse=True)
    fields = ['습관어' , '횟수']
    rows = word_count
    
    with open(r'test1.csv', 'w', newline='\n') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(rows)
        
    test_all_word = pd.read_csv('./test1.csv', encoding='euc-kr')
    
    data1 = morph_list
    cnt = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    morph_list

    for item in data1:
        if item =='아':
            cnt[0] = cnt[0] +1
        elif item == '음':
            cnt[1] = cnt[1] +1
        elif item == '그':
            cnt[2] = cnt[2] +1
        elif item == '어':  
            cnt[3] = cnt[3]+1
        elif item == '그니까':
            cnt[4] = cnt[4] +1
        elif item == '저':
            cnt[5] = cnt[5] +1
        elif item == '이제':
            cnt[6] = cnt[6] +1
        elif item == '막':
            cnt[7] = cnt[7] +1
        elif item == '에':
            cnt[8] = cnt[8] +1
            
    # 습관어 list
    Habitual_word = ['아','음','그','어','그니까','저','이제','막','에']
    
    # 최근 회차 text 결과
    Habitual_word_cnt = cnt
    display(Habitual_word_cnt)
    x = np.arange(len(Habitual_word))
    
    
    f2 = open(output[-2],'r',encoding='euc-kr')
    txt2 = f2.readlines()
    f2.close()
    
    morph_list2 = []
    for i in txt2 :
        morphs = okt.morphs(i)
        for morph in morphs:
            morph_list2.append(morph)
    
    
    def counter(morph_list2):
        word_count2 = {}
        for word in morph_list2:
            if word in word_count2 :
                word_count2[word] += 1
            else :
                word_count2[word] = 1
        return word_count2

    word_count2 = counter(morph_list2)
    
    # 정렬
    word_count2 = sorted(word_count2.items(), key = lambda x:x[1], reverse=True)
    
    fields = ['습관어' , '횟수']
    rows = word_count2
    
    with open(r'test2.csv', 'w', newline='\n') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(rows)
        
    test_all_word2 = pd.read_csv('./test2.csv', encoding='euc-kr')
    
    data2 = morph_list2
    cnt2 = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    
    for item in data2:
        if item =='아':
            cnt2[0] = cnt2[0] +1
        elif item == '음':
            cnt2[1] = cnt2[1] +1
        elif item == '그':
            cnt2[2] = cnt2[2] +1
        elif item == '어':  
            cnt2[3] = cnt2[3]+1
        elif item == '그니까':
            cnt2[4] = cnt2[4] +1
        elif item == '저':
            cnt2[5] = cnt2[5] +1
        elif item == '이제':
            cnt2[6] = cnt2[6] +1
        elif item == '막':
            cnt2[7] = cnt2[7] +1
        elif item == '에':
            cnt2[8] = cnt2[8] +1
    
    # 습관어 list
    Habitual_word = ['아','음','그','어','그니까','저','이제','막','에']

    # 최근 회차 text 결과
    Habitual_word_cnt2 = cnt2
    display(Habitual_word_cnt2)
    x = np.arange(len(Habitual_word))
    
    
    plt.figure(figsize=(10,5))
    plt.rc('font', family='Malgun Gothic')
    plt.xlim(-1,9)
    plt.ylim(0,15)
    p1 = plt.bar(x-0.1, Habitual_word_cnt, width =0.3, color ='yellow',align='center')
    #Habitual_word_cnt2 = [2,3,4,2,5,4,2,3,5]
    p2 = plt.bar(x+0.2, Habitual_word_cnt2, width =0.3, color ='gray',align='center')
    plt.legend((p1[0],p2[0]),('현재결과','직전결과'))

    plt.title('습관어 빈도수 분석결과', fontsize = 20)
    plt.xlabel('<습관어 종류>', fontsize = 14)
    plt.ylabel('<빈도수>', fontsize = 14)
    plt.rc('xtick', labelsize = 12)
    plt.rc('ytick', labelsize = 12)
    #plt.rcParams['figure.figuresize'] = (10,4)
    plt.grid(axis='y', linestyle='--', alpha = 0.5)
    plt.xticks(x, Habitual_word, fontsize=8)

    for i, v in enumerate(x):
        aa = "{:,}회    ".format(Habitual_word_cnt[i])
        plt.text(v,Habitual_word_cnt[i],aa,fontsize=9,color='#997000',
                  horizontalalignment='center',verticalalignment='bottom')

    for i, v in enumerate(x):
        aa = "       {:,}회".format(Habitual_word_cnt2[i])
        plt.text(v,Habitual_word_cnt2[i],aa,fontsize=9,color='black',
                  horizontalalignment='center',verticalalignment='bottom')   


    plt.savefig('./습관어의 종류와 빈도수_비교.png',bbox_inches='tight')
    
    
    return send_file('./습관어의 종류와 빈도수_비교.png')

if __name__ == '__main__':
    app.run(host="localhost",port="9000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9000/ (Press CTRL+C to quit)


step1
['음 저희는 스마트인재개발원에서 자바 기반 웹 서비스 개발자 과정을 교육 받은 교육생들입니다. \n', '\n', '저희 교육 과정에는 음 자바 프로그래밍, 데이터 베이스 실습, 파이썬을 활용하여 데이터 분석 및 시각화 등이 있었으며\n', '\n', "이를 어 활용하여 자바 기반의 웹 서비스에 파이썬을 통한 데이터 분석을 융합하여 '또박또박'이라는 STT 분석 웹 서비스를 프로젝트로 개발하게 되었습니다.    \n", '\n', '자바를 기초로 그니까 사용자 웹 서비스를 구축하고 막 파이썬을 통해 머신러닝과 막 데이터 분석을 산출했습니다. \n']


[0, 2, 0, 1, 1, 0, 0, 2, 1]

[0, 2, 0, 9, 1, 0, 1, 1, 1]

...

127.0.0.1 - - [25/Feb/2021 16:19:54] "GET /end HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 16:19:54] "GET /user?id=admin HTTP/1.1" 200 -


False
False
안녕하세요 저는 김건영입니다
 이게 왜 안 되고 있네 근데 왜 클릭이 안 되지 왜 없어요 그냥 녹으면 되는데


127.0.0.1 - - [25/Feb/2021 16:21:28] "GET /end HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 16:21:29] "GET /user?id=admin HTTP/1.1" 200 -


 이거는 되잖아 이게 클릭되는 무선이 나와야 되는데 왜 이거 이렇게 안 나오지 확인 좀 해
False
False
False
False
안녕 나는 김건영이고 오늘 수업 있을 것 같다 가정 안녕하세요 
안녕 나는 김건영이고 IOT 기반 서비스 개발자 과정 안녕하세요
 아 진짜
 여기 찍어 줘요
여기 찍어 줘요
안 나오네 이거 이미지 못 되나 들어가는 거 안 나오네 이거 이미지 못 되나 들어가는 거

 그게 좀 그런 거 다 대답하면 돼 
 그거 좀 그런 거 다 들어가면 돼 집에서
 왜 잠 안 오면 되는 거 같은데 안 되잖아
 노래 자막 넣으면 되는 거 같은데
 안 되는데
 내려가는 중
 프로에서 내려가는 중
 오늘 저녁 저녁 시간 많이 만들어 놔야지 따로 따로 그거만 누르면 돼
 오늘 저녁 저녁 시간 많이 만들어 놔야지 따로 따로 그거만 녹음을 했는데도 계속 나와요
 했는데도 계속 나와요
 근데 왜 나와
 그거 아니야 지워야 되는데 이거
 그거 아니야 지워야 되는데
 인터넷 사용 기록 지워
 아 몰라
 아 몰라


[2021-02-25 16:26:35,105] ERROR in app: Exception on /user [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\google\api_core\grpc_helpers.py", line 113, in next
    return six.next(self._wrapped)
  File "C:\ProgramData\Anaconda3\lib\site-packages\grpc\_channel.py", line 416, in __next__
    return self._next()
  File "C:\ProgramData\Anaconda3\lib\site-packages\grpc\_channel.py", line 803, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.OUT_OF_RANGE
	details = "Exceeded maximum allowed stream duration of 305 seconds."
	debug_error_string = "{"created":"@1614237995.053000000","description":"Error received from peer ipv4:216.58.197.234:443","file":"src/core/lib/surface/call.cc","file_line":1068,"grpc_message":"Exceeded maximum allowed stream duration of 305 seconds.","grpc_status":11}"
>

The above exception was the direct cause of the following exception:
